# TODOs

* PCA to see which feature has how much influence
* implement scoring function, eating food gives plus, getting smaller distance to food gives plus, reducing distance to enemy wolf gives minus etc.?

In [1]:
import pandas as pd
from sklearn import tree, naive_bayes, svm
import ast
import math
import pickle
import os
import glob

## Use the same config as in the later game

In [2]:
#[General_Constants]
FIELD_WIDTH = 19
FIELD_HEIGHT = 15

#[Game_Constants]
NO_ITERATIONS = 100
MAX_CALC_TIME = 100000

#[Field_Constants]
CELL_EMPTY = '.'
CELL_SHEEP_1 = 'S'
CELL_SHEEP_1_d = 'U'
CELL_WOLF_1 = 'W'
CELL_SHEEP_2 = 's'
CELL_SHEEP_2_d = 'u'
CELL_WOLF_2 = 'w'
CELL_GRASS = 'g'
CELL_RHUBARB = 'r'
CELL_FENCE = '#'


#[Movements]
MOVE_NONE = 0
MOVE_UP = -1
MOVE_DOWN = 1
MOVE_LEFT = -2
MOVE_RIGHT = 2

#[Awards]
AWARD_RHUBARB = 5
AWARD_GRASS = 1

# Load data

Load all games into one dataframe

In [3]:
# path = "../example_player/training_data" 
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0, keep_default_na=False)    
#     df['game'] = filename
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)

In [3]:
path = "../example_player/training_data/subset" 
all_files = glob.glob(path + "/*.csv")   

training_data = []

#load the data into a pandas frames
for file in all_files:
    game_data = pd.read_csv(file,index_col=False)
    reason = game_data.iloc[-1][6]
    
    #if the reason is found, add it to each line to fill out the blanks
    if type(reason) is str:
        for index,row in game_data.iterrows():
            game_data.loc[index,'reason'] = reason

    #else there was no reason, implying the game reached the number of iterations
    else:
        for index,row in game_data.iterrows():
            game_data.loc[index,'reason'] = 'max_iterations'    
    
    training_data.append(game_data)

#preview the final 5 lines
training_data[-1].head()

,field_before,field_after,turn_made_by,move_made,score1,score2,reason
0,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,2,0,0,sheep1 eaten
1,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 sheep,-1,0,0,sheep1 eaten
2,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,2,0,0,sheep1 eaten
3,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 sheep,-1,0,0,sheep1 eaten
4,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 wolf,2,0,0,sheep1 eaten


## Drop unneeded rows

Use only where:
* shows winner movement
* ending reason is '' (ended because of max iterations) or sheep eaten

In [167]:
# df_train = frame[((frame.reason == 'sheep1 eaten') 
#                   | (frame.reason == 'sheep2 eaten')
#                   | (frame.reason == ''))]

# df_train = df_train.join(df_train.groupby('game').agg({'score1':'max','score2':'max'}),on='game',rsuffix='_max')

In [168]:
# def is_winner(row):
#     if '1' in row.turn_made_by:
#         comp= row['score1_max'] > row['score2_max']
#     else:
#         comp= row['score2_max'] > row['score1_max']
#     return comp

# df_train = df_train[df_train.apply(lambda row: is_winner(row),axis=1)]

## Generate general use features

position of wolf, sheep, closest food

In [169]:
# sm = df_train[:10]

In [170]:
# def get_pos(row):    
#     y = 0
#     for r in ast.literal_eval(row['field_before']):
#         x = 0
#         for item in r:
#             if item == 'W':
#                 w1_x = x
#                 w1_y = y
#             elif item == 'S':
#                 s1_x = x
#                 s1_y = y
#             elif item == 'w':
#                 w2_x = x
#                 w2_y = y
#             elif item == 's':
#                 s2_x = x
#                 s2_y = y
#             x += 1
#         y += 1
#     return (w1_x,w1_y,s1_x,s1_y,w2_x,w2_y,s2_x,s2_y)

# sm['pos_'] = sm.apply(lambda row: get_pos(row), axis=1)

/home/chris/PAI/Ex2/venv/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [171]:
# sm[['pos_w1_x', 'pos_w1_y', 'pos_s1_x', 'pos_s1_y', 'pos_w2_x', 'pos_w2_y', 'pos_s2_x', 'pos_s2_y']] = pd.DataFrame(sm['pos_'].tolist(), index=sm.index)  

# Feature selection and Instance selection 

## Sheep

I used a value function for each possible move (up/down/left/right/stay). Value is dependent on how good the position would be (food/distance to wolf etc.). Based on this value function that contains some "spirit" of assignment 1, I train which move should be done to learn from the behavior of the Search Algorithms used in assignment 1.

sheep_coordinates[column,row]

In [85]:
def manhattan_distance(origin, goal):
    return abs(origin[0] - goal[0]) + abs(origin[1] - goal[1])

In [86]:
def valid_move(new_col, new_row, field, player, is_sheep=True):
    # Neither the sheep nor the wolf, can step on a square outside the map. Imagine the map is surrounded by fences.
    if new_row > FIELD_HEIGHT - 1:
        return False
    elif new_row < 0:
        return False
    elif new_col > FIELD_WIDTH - 1:
        return False
    elif new_col < 0:
        return False

    # Neither the sheep nor the wolf, can enter a square with a fence on.
    if field[new_row][new_col] == CELL_FENCE:
        return False

    if is_sheep:
        # Sheep can not step on squares occupied by the wolf of the same player.
        # Sheep can not step on squares occupied by the opposite sheep.
        if '1' in player:
            if field[new_row][new_col] == CELL_SHEEP_2 or \
                    field[new_row][new_col] == CELL_WOLF_1 or \
                    field[new_row][new_col] == CELL_WOLF_2:
                return False
        else:
            if field[new_row][new_col] == CELL_SHEEP_1 or \
                    field[new_row][new_col] == CELL_WOLF_2 or \
                    field[new_row][new_col] == CELL_WOLF_1:
                return False
    else:
        # Wolfs can not step on squares occupied by the opponents wolf (wolfs block each other).
        # Wolfs can not step on squares occupied by the sheep of the same player .
        if '1' in player:
            if field[new_row][new_col] == CELL_WOLF_2:
                return False
            elif field[new_row][new_col] == CELL_SHEEP_1:
                return False
        else:
            if field[new_row][new_col] == CELL_WOLF_1:
                return False
            elif field[new_row][new_col] == CELL_SHEEP_2:
                return False
    return True

In [87]:
def value_of_move(new_col, new_row, sheep_position, wolf_position, field, food_goal, player, is_sheep=True):
    if valid_move(new_col, new_row, field, player, is_sheep):
        move_value = 0

        if manhattan_distance((new_col, new_row), wolf_position) < 3:
            move_value -= 50

        if food_goal:
            # set direction of food
            sf_col_food_before = abs(sheep_position[0] - food_goal[0])
            sf_row_food_before = abs(sheep_position[1] - food_goal[1])
            sf_col_food_after = abs(new_col - food_goal[0])
            sf_row_food_after = abs(new_row - food_goal[1])

            if sf_row_food_after > sf_row_food_before:
                move_value -= 3
            elif sf_row_food_after < sf_row_food_before:
                move_value += 2
            if sf_col_food_after > sf_col_food_before:
                move_value -= 3
            elif sf_col_food_after < sf_col_food_before:
                move_value += 2

        return move_value
    else:
        # never even attempt to go there
        return -1000

In [96]:
X_sheep = []
Y_sheep = []
number_moves = 0

for game in training_data:
    
    #we want to learn from the winning player, which is the player with the highest score:
    if game.iloc[-1][4] < game.iloc[-1][5]:
        sheep = 's'
        wolf = 'W'
    
    elif game.iloc[-1][4] > game.iloc[-1][5]:
        sheep = 'S'
        wolf = 'w'
    else:
        continue

    rhubarb = 'r'
    grass = 'g'

    #for each game state in our training data
    for index,row in game.iterrows():

        #we don't want games that ended because of an error or because the sheep commited suicide
        if row['reason'] not in ('sheep1 eaten','sheep2 eaten','max_iterations'):
            continue

        #we want to only learn from sheep
        if row['move_made'] == 'player1 wolf' or row['move_made'] == 'player2 wolf':
            continue
        
        number_moves += 1
        
        #this is the move that we are learning from this game state
        move = row['move_made']

        #create empty feature array for this game state
        game_features = []

        #turn the field from before the move from a string back to a list
        field= ast.literal_eval(row['field_before'])

        #get positions of sheep, wolf and food items
        food = []
        y=0
        for field_row in field:
            x = 0
            for item in field_row:
                if item == sheep:
                    sheep_position = (x, y)
                elif item == wolf:
                    wolf_position = (x, y)
                elif item == rhubarb or item == grass:
                    food.append((x, y))
                x += 1
            y += 1

        # feature 1: x-distance wolf
        sf_x_wolf = sheep_position[0] - wolf_position[0]
        # feature 2: y-distance wolf
        sf_y_wolf = sheep_position[1] - wolf_position[1]

        # determine closest food:
        food_distance = 1000
        food_goal = None
        for food_item in food:
            distance = abs(food_item[0] - sheep_position[0]) + abs(food_item[1] - sheep_position[1])
            if distance < food_distance:
                food_distance = distance
                food_goal = food_item

        # feature 3: x-distance to food
        sf_x_food = 0
        # feature 4: y-distance to food
        sf_y_food = 0               
        if food_goal:
            sf_x_food = sheep_position[0] - food_goal[0]
            sf_y_food = sheep_position[1] - food_goal[1]

#         # feature 5: is going up allowed?
#         sf_up = valid_move(sheep_position[0] - 1, sheep_position[1], field,
#                            row['turn_made_by'], True)
#         # feature 6: is going down allowed?
#         sf_down = valid_move(sheep_position[0] + 1, sheep_position[1], field,
#                              row['turn_made_by'], True)
#         # feature 7: is going right allowed?
#         sf_right = valid_move(sheep_position[0], sheep_position[1] + 1, field,
#                               row['turn_made_by'], True)
#         # feature 8: is going left allowed?
#         sf_left = valid_move(sheep_position[0], sheep_position[1] - 1, field,
#                              row['turn_made_by'], True)
        # feature: value of moving left
        sf_v_left = value_of_move(sheep_position[0] -1, sheep_position[1], sheep_position, wolf_position, field, food_goal,
                           row['turn_made_by'], True)
        # feature: value of moving up
        sf_v_up = value_of_move(sheep_position[0], sheep_position[1] - 1, sheep_position, wolf_position, field, food_goal,
                           row['turn_made_by'], True)
        # feature: value of not moving
        sf_v_stay = value_of_move(sheep_position[0], sheep_position[1], sheep_position, wolf_position, field, food_goal,
                           row['turn_made_by'], True)
        # feature: value of moving down
        sf_v_down = value_of_move(sheep_position[0], sheep_position[1] + 1, sheep_position, wolf_position, field, food_goal,
                           row['turn_made_by'], True)
        # feature: value of moving right
        sf_v_right = value_of_move(sheep_position[0] + 1, sheep_position[1], sheep_position, wolf_position, field, food_goal,
                           row['turn_made_by'], True)
      
        
#         game_features.append(sf_x_wolf)
#         game_features.append(sf_y_wolf)
#         game_features.append(sf_x_food)
#         game_features.append(sf_y_food)
#         game_features.append(sf_up)
#         game_features.append(sf_down)
#         game_features.append(sf_right)
#         game_features.append(sf_left)
        game_features.append(sf_v_left)
        game_features.append(sf_v_up)
        game_features.append(sf_v_stay)
        game_features.append(sf_v_down)
        game_features.append(sf_v_right)

        #add features and move to X_sheep and Y_sheep
        X_sheep.append(game_features)
        Y_sheep.append(move)

In [97]:
#this prints an example of our feature and outcome vector:
print(X_sheep[:10])
print(Y_sheep[:10])
print(number_moves)

[[-3, -3, 0, 2, 2], [-3, -3, -1000, 2, 2], [-3, -3, 0, 2, 2], [-3, -3, -1000, 2, 2], [-3, -3, 0, 2, 2], [-3, -3, -1000, 2, 2], [-3, -3, 0, 2, 2], [-3, -3, -1000, 2, 2], [-3, -3, 0, 2, 2], [-3, -3, -1000, 2, 2]]
[2, -1, 2, -1, 1, -1, 2, -1, 2, -1]
2992


In [98]:
df_investigate = pd.DataFrame(X_sheep)
df_investigate['y'] = Y_sheep
df_investigate

,0,1,2,3,4,y
0,-3,-3,0,2,2,2
1,-3,-3,-1000,2,2,-1
2,-3,-3,0,2,2,2
3,-3,-3,-1000,2,2,-1
4,-3,-3,0,2,2,1
5,-3,-3,-1000,2,2,-1
6,-3,-3,0,2,2,2
7,-3,-3,-1000,2,2,-1
8,-3,-3,0,2,2,2
9,-3,-3,-1000,2,2,-1


In [99]:
Y_sheep_f = (pd.DataFrame(X_sheep).idxmax(axis=1) - 2).values

In [100]:
# # we want to only learn from sheep
# df_sheep = df_train[((df_train.turn_made_by == 'player1 sheep') | (df_train.turn_made_by == 'player2 sheep'))]

# df_sheep['f1'] = 

this prints an example of our feature and outcome vector:

In [101]:
# print(X_sheep[0])
# print(Y_sheep[0])
# print(number_moves)

## Wolf

Explain here in text which feature you used for the wolf, and which data you used for training.

In [102]:
#construct your features here

# Build Sheep Model

Min-Max-normalizing for all values

In [103]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# X_sheep_norm=scaler.fit_transform(X_sheep)

PCA (https://stackoverflow.com/questions/50796024/feature-variable-importance-after-a-pca-analysis)

In [104]:
# from sklearn import decomposition
# pca = decomposition.PCA(n_components=3)
# pca.fit(X_sheep_norm)
# print(pca.explained_variance_ratio_)
# print(abs( pca.components_ ))


Split into train and test data

In [105]:
# X_sheep

In [106]:
from sklearn import model_selection
x_train_s, x_test_s, y_train_s, y_test_s = model_selection.train_test_split(X_sheep, Y_sheep, test_size=0.25)
# x_train_s, x_test_s, y_train_s, y_test_s = model_selection.train_test_split(X_sheep, Y_sheep_f, test_size=0.25)

## Train & Test sheep

Allowed algorithms:
* Naive Bayes (GaussianNB, MultinominalNB, ComplementNB, BernoulliNB)
* Decision Tree
* Support Vector Machine (SVC, NuSVC, LinearSVC)

In [107]:
from sklearn.metrics import confusion_matrix
import numpy as np

## Naive Bayes

In [108]:
# sheep_NB = naive_bayes.GaussianNB()
# # sheep_NB = naive_bayes.MultinomialNB()
# # sheep_NB = naive_bayes.ComplementNB()
# # sheep_NB = naive_bayes.BernoulliNB()
# sheep_NB = sheep_NB.fit(x_train_s,y_train_s)
# y_pred_s = sheep_NB.predict(x_test_s)

# cm = confusion_matrix(y_test_s, y_pred_s.tolist())
# recall = np.diag(cm) / np.sum(cm, axis = 1)
# precision = np.diag(cm) / np.sum(cm, axis = 0)

# print(cm)
# print(np.mean(recall))
# print(np.mean(precision))

## Decision Tree

In [109]:
sheep_tree = tree.DecisionTreeClassifier()
sheep_tree = sheep_tree.fit(x_train_s,y_train_s)
y_pred_s = sheep_tree.predict(x_test_s)

cm = confusion_matrix(y_test_s, y_pred_s.tolist())
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)

print(cm)
print(np.mean(recall))
print(np.mean(precision))

[[73 15 23 11 19]
 [33 52 40 22 40]
 [ 8  6 47  4 16]
 [32  9 28 72 24]
 [15  8 38 37 76]]
0.4498395043806693
0.4451351561217553


## SVM

In [110]:
# sheep_svm = svm.SVC()
# # sheep_svm = svm.NuSVC()
# # sheep_svm = svm.LinearSVC()
# sheep_svm = sheep_svm.fit(x_train_s,y_train_s)
# y_pred_s = sheep_svm.predict(x_test_s)

# cm = confusion_matrix(y_test_s, y_pred_s.tolist())
# recall = np.diag(cm) / np.sum(cm, axis = 1)
# precision = np.diag(cm) / np.sum(cm, axis = 0)

# print(cm)
# print(np.mean(recall))
# print(np.mean(precision))

# Train wolf

In [111]:
# wolf_tree = tree.DecisionTreeClassifier()
# wolf_tree = wolf_tree.fit(X_wolf,Y_wolf)

# Save models to files

Save your models to files here using pickle. Change the [uzhshortname] to your own UZH shortname. This name needs to match the model that you caller in your python player file.

In [112]:
sheep_filename = 'chriweb_sheep_model.sav'
# wolf_filename = 'chriweb_wolf_model.sav'

pickle.dump(sheep_tree,open(sheep_filename,'wb'))
# pickle.dump(wolf_tree,open(wolf_filename,'wb'))